# Implementation of the `scaredyFish` model in Julia
Simulate a spreading process among agents with information transfer

## 1. Define our functions our important functions, and run one simulation

### 1a. Let's first define all of our functions that do the work

In [1]:
## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
## Master function to loop through time steps and give us a table of results
## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 

## pre
__precompile__()

# init
using Distributions, Distances, IterTools, DelimitedFiles

## start function
function scaredyFish( ; 
        agents, 
        maxtime, 
        walk, 
        run, 
        drift, 
        strengthExcitation, 
        strengthInhibition, 
        sensory, 
        trigger, 
        hideProb )
    
    ## -
    ## Prep simulation, define some empty arrays to fill in
    ## -

    ## make blank results array and initialize with starting location
    r = makeBlankResultsArray( maxtime = maxtime::Int64, agents = agents::Int64 )

    ## make a vector that keeps track of dissappeared agents
    gone = zeros( agents )
    
    ## pre-select your false starter
    falseStarter = first( sample( 1 : agents, 1 ) )
    
    
    ## ---
    ## Start workhorse
    ## ---
    
    ## for t in ___time___
    @inbounds for t in 2 : maxtime
        
        ## -
        ## Here starts the information updating section:
        ## for each agent, we look at the sensory input it recieved in the last time step
        ## and update our position and state accordingly
        ## -
        
        ## extract current coordinates
        C = hcat( r[ t - 1, 3, : ], r[ t - 1, 4, : ] )
        
        ## call neighborhood search function, makes adjacency matrix of information exchanges
        D = neighborSearch( C = C::Array, sensory = sensory::Float64 )
        
        
        ## -
        ## Here starts the state and then position updating section
        ## -
        
        ## for a in ___agents____, calculate decision function and update state
        r = agentStateStepUpdate( 
            agents = agents::Int64, 
            maxtime = maxtime::Int64,
            gone =  gone::Array, 
            D = D::Array, 
            strengthExcitation = strengthExcitation::Float64, 
            strengthInhibition = strengthInhibition::Float64, 
            r = r::AbstractArray, 
            t = t::Int64, 
            hideProb = hideProb::Float64,
            walk = walk::Float64, 
            run = run::Float64, 
            drift = drift::Float64,
            trigger = trigger::Int64,
            falseStarter = falseStarter::Int64 
            )
        
        ##
        ## kill the simulation if all agents have calmed down
        ##
        if all( r[ t, 5, : ] .== 0 ) && t > ( trigger + 2 )

            ## subset output
            r = r[ 1 : t, :, : ]

            ## break loop
            break

        end
          
    end
    
    ## return results
    return r
    
end


## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
## Function that makes a blank results array to be filled in
## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
function makeBlankResultsArray( ; maxtime, agents )
    
    ## empty 3D array to populate ( time x number of variables x agents )
    ## my columns are 
    ## | 1. time 
    ## | 2. agent id 
    ## | 3. x 
    ## | 4. y 
    ## | 5. state 0 (feed) or 1 (flee)
    ## | 6. in a white tile or not
    r = zeros( maxtime, 6, agents )
    
    ## populate first row of each matrix
    @inbounds Threads.@threads for agent in 1 : size( r, 3 )

        ## put a random number in the first row
        r[ 1, 1, agent ] = 1
        r[ 1, 2, agent ] = agent
        
        ## sample a white tile
        sampledTile = whiteTiles[ sample( 1 : size( whiteTiles )[ 1 ], 1 ), : ]
        
        ## put agent somewhere in this sampled white tile
        r[ 1, 3, agent ] = sampledTile[ 1 ] + Float64.( rand( Uniform( -0.5, 0.5 ) ) )
        r[ 1, 4, agent ] = sampledTile[ 2 ] + Float64.( rand( Uniform( -0.5, 0.5 ) ) )
        
        ## fill in state and location
        r[ 1, 5, agent ] = 0::Int64
        r[ 1, 6, agent ] = 1::Int64

    end
    
    ## return
    return r
    
end


## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
## Function to udpdate agent state after calculating decision function, then update position accordingly
## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
function agentStateStepUpdate( ; agents, maxtime, gone, D, strengthExcitation, strengthInhibition, r, t, hideProb, walk, run, drift, trigger, falseStarter )
    
    ## loop through agents
    @inbounds Threads.@threads for a in 1 : agents
           
        ## skip if its a hider
        if gone[ a ] == 1

            continue

        end
        
        ## add time info to output table
        r[ t, 1, a ] = t

        ## add agent id to output table
        r[ t, 2, a ] = a
        
        ## grab the states at time t - 1
        currStates = r[ t - 1, 5, : ]

        ## excitation information
        excitation = strengthExcitation * sum( D[ a, findall( currStates .== 1 ) ] )

        ## inhibition information
        inhibition = strengthInhibition * sum( D[ a, findall( currStates .== 0 ) ] )

        ## decision making function
        decision = excitation - inhibition

        ## if we pass threshold
        if decision > 1

            ## update state
            r[ t, 5, a ] = 1

        end

        ## force the agent to rest if its been fleeing for 2 timesteps, or
        ## potentially hide if its on a black tile
        if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 1

            ## if you're on a black tile, flip a coin to hide or rest
            if r[ t, 6, a ] == 0

                ## flip a coin
                hide = rand( Bernoulli( hideProb ), 1 )

                ## if you're hiding, teleport to -1e6 forever
                if hide == Bool[ 1 ]

                    ## update state
                    r[ t, 5, a ] = 0

                    ## teleport
                    r[ t : maxtime, 3, a ] .= -1e6
                    r[ t : maxtime, 4, a ] .= -1e6

                    ## update gone
                    gone[ a ] = 1

                end

            ## if you're on a white tile, force a rest time step
            else

                ## update state
                r[ t, 5, a ] = 0

            end

        end
        
        ## ~ ~ ~
        ## trigger false alarm
        ## ~ ~ ~
        
        ## initiate
        if t == trigger && a == falseStarter

            ## switch state
            r[ t, 5, a ] = 1

        end
        
        
        ## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        ## step updating section
        ## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
        
        ## -
        ## first thing we need to know is where the nearest white tile is
        ## -

        ## grab my previous x and y before calculating updated step
        currX = r[ t - 1, 3, a ]
        currY = r[ t - 1, 4, a ]

        ## round them to find the nearest tile regardless of color
        currPatchX = round.( Int, currX )
        currPatchY = round.( Int, currY )

        ## check that it is actually a tile, and you are not out of bounds
        if ( 0.5 < currPatchX < ( 0.5 + nrows ) ) && ( 0.5 < currPatchY < ( 0.5 + ncols ) )

            ## note that I am in bounds
            inBounds = 1

            ## if this is a white tile, we are already done
            if landscape[ currPatchX, currPatchY ] == 1

                ## make this my home patch
                homePatchX = currPatchX
                homePatchY = currPatchY

                ## record status in output ( 1 if I'm in a white tile, 0 if not )
                r[ t, 6, a ] = 1

            ## otherwise, find the nearest white tile
            else

                ## distances to white tiles
                toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

                ## which min
                homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                ## record status in output
                r[ t, 6, a ] = 0

            end

        ## if I'm out of bounds...
        else

            ## note it
            inBounds = 0

            ## distances to white tiles
            toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

            ## which min
            homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
            homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

            ## record status in output
            r[ t, 6, a ] = 0

        end


        ## -
        ## Here starts the step updating section
        ## -

        ## if feeding and in bounds
        if r[ t, 5, a ] == 0 && inBounds == 1

            ## random walk steps
            r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, walk ) )
            r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, walk ) )

            ## go to next agent
            continue

        end

        ## if feeding and out of bounds
        if r[ t, 5, a ] == 0 && inBounds == 0

            ## bias my walk in this direction
            xBias = ( homePatchX - r[ t - 1, 3, a ] )
            yBias = ( homePatchY - r[ t - 1, 4, a ] )

            ## random walk steps
            r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( xBias * drift, walk ) )
            r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( yBias * drift, walk ) )

            ## go to next agent
            continue

        end

        ## if starting fleeing
        if r[ t, 5, a ] == 1 && r[ t - 1, 5, a ] == 0

            ## random walk steps
            r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, run ) )
            r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, run ) )

            ## go to next agent
            continue

        end

        ## if continuing fleeing
        if r[ t, 5, a ] == 1 && r[ t - 1, 5, a ] == 1

            ## bias my walk in this direction
            xBias = ( r[ t - 1, 3, a ] - r[ t - 2, 3, a ] )
            yBias = ( r[ t - 1, 4, a ] - r[ t - 2, 4, a ] )

            ## random walk steps
            r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( xBias, 0.01 ) )
            r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( yBias, 0.01 ) )

        end     
        
    end
    
    return r
    
end



## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
## a function to calculate thresholded adjacency matrix through neighbor search
## input: a 2-column matrix of current agent x and y coords
## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
function neighborSearch( ; C, sensory )

    ## calculate rounded coordinates
    roundC = round.( C )
    
    ## get n rows
    rowsC = size( roundC )[ 1 ]
    
    ## blank distance matrix to fill in
    blank = zeros( rowsC, rowsC )

    ## loop through and fill in appropriately
    @inbounds Threads.@threads for ind in 1 : rowsC

        ## currX and Y
        realX = C[ ind, 1 ]
        realY = C[ ind, 2 ]

        ## select the first for, as my current
        tempX = roundC[ ind, 1 ]
        tempY = roundC[ ind, 2 ]
        
        ## neighborhood size
        nSize = round( sensory + 1 )

        ## range to look over
        loX = tempX - nSize
        hiX = tempX + nSize
        loY = tempY - nSize
        hiY = tempY + nSize

        ## get indices of in-range agents
        getThese = findall( 
            ( roundC[ :, 1 ] .> loX ) .& 
            ( roundC[ :, 1 ] .< hiX ) .& 
            ( roundC[ :, 2 ] .> loY ) .& 
            ( roundC[ :, 2 ] .< hiY ) )

        ## remove self calculation
        getThese = deleteat!( getThese, getThese .== ind )

        ## get distances to current x and y
        neighborhood = colwise( Euclidean(), [ realX, realY ], transpose( C[ getThese, : ] ) )

        ## fill in my blank matrix
        blank[ ind, getThese ] = neighborhood
        blank[ getThese, ind ] = neighborhood

    end;

    ## threshold the final product
    blank[ blank .> sensory ] .= 0

    ## convert to 1 / D ^ 2
    blank[ blank .> 0 ] = 1 ./ blank[ blank .> 0 ] .^ 2
    
    ## return blank
    return blank
    
end


## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
## a function to flatten a 3d array, R equivalent of do.call( rbind )
## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
function flatten( ; data ) 
    
    s = data[ :, :, 1 ]
    
    ## loop through and append stack
    for i in 2 : size( data, 3 )
    
        ## add
        s = vcat( s, data[ :, :, i ] )
    
    end
    
    ## output
    return s
    
end


## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
## timed master function
## ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
timed_scaredyFish( ; 
    agents, 
    maxtime, 
    walk, 
    run, 
    drift, 
    strengthExcitation, 
    strengthInhibition, 
    sensory, 
    trigger, 
    hideProb ) = @time scaredyFish( ; 
        agents, 
        maxtime, 
        walk, 
        run, 
        drift, 
        strengthExcitation, 
        strengthInhibition, 
        sensory, 
        trigger, 
        hideProb ) ;


### 1b. Load spatial landscape
This prepares our spatial domain

In [2]:
## -
## Define my landscape
## -

# ## number of rows and cols
# nrows = 20
# ncols = 100
#
# ## landscape is a binary matrix, where 1s are foraging patches (white tiles)
# landscape = rand( Bernoulli( 0.5 ), nrows, ncols )

## read table
global landscape = readdlm( "reef.tsv", '\t' )

## dimensions
global nrows = size( landscape )[ 1 ]
global ncols = size( landscape )[ 2 ]

## indices for white tiles
global whiteTiles = getindex.( findall( y -> y == 1, landscape ), [ 1 2 ] ) ;

### 1c. Run one simulation
Pass parameter values to our function and let it rip

In [4]:
## ---
## run my simulation once
## ---
r = timed_scaredyFish(
    agents = 400,
    maxtime = 100,
    walk = 0.1,
    run = 0.5,
    drift = 0.2,
    strengthExcitation = 5.0,
    strengthInhibition = 0.25,
    sensory = 2.5,
    trigger = 4,
    hideProb = 0.5
    )

## print size
size( r )

  0.081148 seconds (68.00 k allocations: 99.610 MiB)


(7, 6, 400)

### 1d. Prep my one simulation for writing and save
Flatten my array from 3D to 2D

In [5]:
## flatten
one = flatten( data = r )

# write output
writedlm( "out.tsv", one, '\t') ;

## 2. Run a bunch of simulations
Do a bunch of these simulations and save summary statistics

In [7]:
## ---
## Set up sweep file
## ---
sweep = vcat( fill.( [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2 ], 100 )... )

## loop function
__precompile__() ; begin
    
    function multSimulations( ; sweep )

        ## loop
        for s in 1 : size( sweep )[ 1 ] 

            ## set current inhibition
            currInhibition = sweep[ s ]

            ## run a sim
            results = scaredyFish(
                agents = 4000,
                maxtime = 1000,
                walk = 0.1,
                run = 0.5,
                drift = 0.25,
                strengthExcitation = 5.0,
                strengthInhibition = currInhibition,
                sensory = 2.5,
                trigger = 4,
                hideProb = 0.5
                )
            
            ## flatten r
            flatr = flatten( data = results )
            
            ## append simulation number column
            flatr = hcat( flatr, vcat( fill.( [ s ], size( flatr )[ 1 ] )... ) )
            
            ## append inhibition value
            flatr = hcat( flatr, vcat( fill.( [ currInhibition ], size( flatr )[ 1 ] )... ) )
            
            ## attach
            if s == 1
                
                ## start my master file
                tempResult = copy( flatr )
                global tempResult
                
            ## else
            else
                
                ## append
                tempResult = vcat( tempResult, flatr )
                global tempResult
                
            end

            ## print size
            print( s, size( flatr ), " -- " )

        end
        
        ## return
        return tempResult

        end
    
end

## run 
master = multSimulations( sweep = sweep::Array )

# write output
writedlm( "results.tsv",  master, '\t') ;


1(540000, 8) -- 2(776000, 8) -- 3(28000, 8) -- 4(32000, 8) -- 5(384000, 8) -- 6(28000, 8) -- 7(660000, 8) -- 8(468000, 8) -- 9(444000, 8) -- 10(528000, 8) -- 11(584000, 8) -- 12(500000, 8) -- 13(496000, 8) -- 14(28000, 8) -- 15(480000, 8) -- 16(724000, 8) -- 17(36000, 8) -- 18(520000, 8) -- 19(60000, 8) -- 20(572000, 8) -- 21(352000, 8) -- 22(76000, 8) -- 23(576000, 8) -- 24(600000, 8) -- 25(464000, 8) -- 26(604000, 8) -- 27(444000, 8) -- 28(880000, 8) -- 29(252000, 8) -- 30(268000, 8) -- 31(664000, 8) -- 32(564000, 8) -- 33(640000, 8) -- 34(504000, 8) -- 35(60000, 8) -- 36(764000, 8) -- 37(568000, 8) -- 38(680000, 8) -- 39(680000, 8) -- 40(744000, 8) -- 41(64000, 8) -- 42(572000, 8) -- 43(824000, 8) -- 44(544000, 8) -- 45(564000, 8) -- 46(704000, 8) -- 47(772000, 8) -- 48(704000, 8) -- 49(576000, 8) -- 50(892000, 8) -- 51(600000, 8) -- 52(664000, 8) -- 53(832000, 8) -- 54(584000, 8) -- 55(40000, 8) -- 56(564000, 8) -- 57(772000, 8) -- 58(656000, 8) -- 59(1056000, 8) -- 60(784000, 8) -

In [1]:
Threads.nthreads()

22